In [ ]:
from polyglot.detect import Detector
from polyglot.detect.base import logger as polyglot_logger
import pandas as pd
from pyspark.sql import SparkSession
#import findspark

In [ ]:
polyglot_logger.setLevel("ERROR")

## Create Session

In [ ]:
spark = SparkSession.builder.master('local[*]').config("spark.driver.memory", "16g").appName("ML2").getOrCreate()

23/01/24 23:38:53 WARN Utils: Your hostname, ares resolves to a loopback address: 127.0.1.1; using 192.168.1.181 instead (on interface enp4s0)
23/01/24 23:38:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/24 23:38:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Load data

In [ ]:
df_review = spark.read.json('./item_dedup.json')
df_review.createOrReplaceTempView('review')
df_article = spark.read.json('./metadata.json')
df_article.createOrReplaceTempView('article')

## View

In [ ]:
df_review.show(n = 10)

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|0000000078|  [1, 1]|    5.0|Conversations wit...|08 11, 2004|A3AF8FFZAZYNE5|                null|          Impactful!|    1092182400|
|0000000116|  [5, 5]|    4.0|Interesting Grish...|04 27, 2002| AH2L9G3DQHHAJ|               chris|  Show me the money!|    1019865600|
|0000000116|  [0, 0]|    1.0|The thumbnail is ...|03 24, 2014|A2IIIDRK3PRRZY|              Helene|Listing is all sc...|    1395619200|
|0000000868|[10, 10]|    4.0|I'll be honest. I...|09 11, 2002|A1TADCM7YWPQ8M|            Joel@AWS|Not a Bad Transla...|    1031702400|
|0000013714|  [0, 0]|    4.0|It had all the so...|10 31

In [ ]:
df_article.show(n=10)

+---------------+----------+----------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+
|_corrupt_record|      asin|     brand|          categories|         description|               imUrl|price|             related|           salesRank|               title|
+---------------+----------+----------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+
|           null|0001048791|      null|           [[Books]]|                null|http://ecx.images...| null|                null|{null, null, null...|The Crucible: Per...|
|           null|0000143561|      null|[[Movies & TV, Mo...|3Pack DVD set - I...|http://g-ecx.imag...|12.99|{null, [B0036FO6S...|{null, null, null...|Everyday Italian ...|
|           null|0000037214|Big Dreams|[[Clothing, Shoes...|                null|http://ecx.images...| 6.99|{null, [B00JO8II7...|{null, null

In [ ]:
df_sample_review = df_review.sample(0.0101, 1)

In [ ]:
df_sample_review.createOrReplaceTempView('review_sample')

In [ ]:
res = spark.sql('SELECT a.title, s.* FROM review_sample s INNER JOIN (SELECT asin, title FROM article ORDER BY asin) AS a ON s.asin = a.asin')

In [ ]:
res.show(n=10)

+--------------------+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|               title|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+--------------------+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|Adult Ballet Tutu...|0000032034|[11, 12]|    5.0|Purchased and wor...|09 10, 2013|A2MPIVS0KSY8O0|               Sonja|               Tutus|    1378771200|
| Along Came a Spider|0006476155|  [0, 0]|    5.0|Great development...|01 14, 2013|A37QDGQ90TLQQX|     Albert Wimberly|Great character d...|    1358121600|
| Along Came a Spider|0006476155|  [0, 0]|    4.0|Always a true fan...| 01 5, 2014|A3S8MFLN2HZ7RI|          footsie517|          Love it!!!|    1388880000|
| Along Came a Spider|0006476155|  [0, 0]|    5.0|I enjoy a good

In [ ]:
df_sample = res.toPandas()

In [ ]:
df_sample.head(3)

,title,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,Adult Ballet Tutu Yellow,0000032034,"[11, 12]",5.0,Purchased and wore these tutus for the Portlan...,"09 10, 2013",A2MPIVS0KSY8O0,Sonja,Tutus,1378771200
1,Along Came a Spider,0006476155,"[0, 0]",5.0,"Great development of characters, you begin to ...","01 14, 2013",A37QDGQ90TLQQX,Albert Wimberly,Great character development / compelling case ...,1358121600
2,Along Came a Spider,0006476155,"[0, 0]",4.0,Always a true fan of James Patterson. He knows...,"01 5, 2014",A3S8MFLN2HZ7RI,footsie517,Love it!!!,1388880000


In [ ]:
try:
  df_sample.drop(columns=['as)n'], inplace=True)
except:
  pass

In [ ]:
df_sample.head(3)

,title,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,Adult Ballet Tutu Yellow,0000032034,"[11, 12]",5.0,Purchased and wore these tutus for the Portlan...,"09 10, 2013",A2MPIVS0KSY8O0,Sonja,Tutus,1378771200
1,Along Came a Spider,0006476155,"[0, 0]",5.0,"Great development of characters, you begin to ...","01 14, 2013",A37QDGQ90TLQQX,Albert Wimberly,Great character development / compelling case ...,1358121600
2,Along Came a Spider,0006476155,"[0, 0]",4.0,Always a true fan of James Patterson. He knows...,"01 5, 2014",A3S8MFLN2HZ7RI,footsie517,Love it!!!,1388880000


In [ ]:
df_sample.shape

(672885, 10)

In [ ]:
df_train = df_sample.iloc[0:int(df_sample.shape[0] * 0.99),:]

In [ ]:
df_train.shape

(666156, 10)

In [ ]:
df_test = df_sample.iloc[int(df_sample.shape[0] * 0.99):df_sample.shape[0],:]

In [ ]:
df_test.shape

(6729, 10)

In [ ]:
df_train.to_csv('./sample.csv', sep=';', index=False)

In [ ]:
df_test.to_csv('./test.csv', sep=';', index=False)